# Import

In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import os
import matplotlib.pyplot as plt
import statistics as st

In [2]:
def scrapper(M,Y):
    url= "https://en.tutiempo.net/climate/"+ M +"-"+ Y +"/ws-427540.html"

    req = requests.get(url)

    soup = BeautifulSoup(req.text, "html.parser")

    print(soup.select("h3",class_="bluecab")[0].text)

    all=[]

    for row in (soup.select('table')[3]).select("tr"):
        row1 = []
        if(len(row.text)>10):
            for i in row.select("td"):
                row1.append(i.text)
            all.append(row1)


        df = pd.DataFrame(all[1:])


    dict = {'date':df[0],'T':df[1] ,'TM':df[2],'Tm':df[3],'SLP':df[4],'H':df[5],'PP':df[6],'VV':df[7],'V':df[8],'VM':df[9]}
    df = pd.DataFrame(dict)
    
    return df

    

In [3]:
def data_cleaner(df):
    df.dropna(how='all',inplace = True)

    index = pd.Index(list(range(0, len(df.index),1)))
    df = df.set_index(index)
    

    missing_Val = []
    

    for r in df.index:
        if(df.iloc[r,0] == '\xa0' ):
            missing_Val.append(r)
    df.drop(missing_Val,inplace=True)       
    # missing_Val

    for c in range(len(df.columns)):

        for r in range(len(df.index)):
            if(df.iloc[r,c] == '-' ):
                temp = []
                for i in df.iloc[:,c]:
                    try:
                        temp.append(float(i))
                    except:
                        pass
                df.iloc[r,c] = st.mean(temp)
                
    df.iloc[:,1:] = df.iloc[:,1:].astype(float)
    df.iloc[:,0] = df.iloc[:,0].astype(int)
    
    return df

In [4]:
def add_month(final):
    
    month = []
    a=0
    for i in range(len(final.index)):
        if (final.iloc[i,0] == 1):
            a=a+1
        month.append(a)

    final['Month'] = month
    
    return final


In [5]:
def reset_index(data):    
    index = pd.Index(list(range(0, len(data.index),1)))
    data = data.set_index(index)
    return data

In [7]:
def all_month_in_Year(Y):

    final = pd.DataFrame()
    date = ['01', '02', '03', '04', '05', '06', '07', '08', '09', '10', '11', '12']

    for M in date:
        df = scrapper(M,Y)
        df = data_cleaner(df)
        final = pd.concat([final,df],axis=0)

    final = reset_index(final)

    final = add_month(final)

    return final

In [48]:
def add_years(all_years):
    yr_list = [] 
    yr = 1972
    for i in range(len(all_years.index)):
        if all_years.index[i] == 0:
            yr = yr+1
        yr_list.append(yr)
    yr_list


    all_years['years'] = yr_list 
    
    return all_years

In [16]:
all_years = pd.DataFrame()
years = [str(x) for x in range(1973,2021 + 1,1)]

for year in years:
    all_years = pd.concat([all_years,all_month_in_Year(year)],axis=0)
    
all_years = add_years(all_years)
all_years = reset_index(all_years)


Climate data: January 1973
Climate data: February 1973
Climate data: March 1973
Climate data: April 1973
Climate data: May 1973
Climate data: June 1973
Climate data: July 1973
Climate data: August 1973
Climate data: September 1973
Climate data: October 1973
Climate data: November 1973
Climate data: December 1973
Climate data: January 1974
Climate data: February 1974
Climate data: March 1974
Climate data: April 1974
Climate data: May 1974
Climate data: June 1974
Climate data: July 1974
Climate data: August 1974
Climate data: September 1974
Climate data: October 1974
Climate data: November 1974
Climate data: December 1974
Climate data: January 1975
Climate data: February 1975
Climate data: March 1975
Climate data: April 1975
Climate data: May 1975
Climate data: June 1975
Climate data: July 1975
Climate data: August 1975
Climate data: September 1975
Climate data: October 1975
Climate data: November 1975
Climate data: December 1975
Climate data: January 1976
Climate data: February 1976
Cli

Climate data: March 1999
Climate data: April 1999
Climate data: May 1999
Climate data: June 1999
Climate data: July 1999
Climate data: August 1999
Climate data: September 1999
Climate data: October 1999
Climate data: November 1999
Climate data: December 1999
Climate data: January 2000
Climate data: February 2000
Climate data: March 2000
Climate data: April 2000
Climate data: May 2000
Climate data: June 2000
Climate data: July 2000
Climate data: August 2000
Climate data: September 2000
Climate data: October 2000
Climate data: November 2000
Climate data: December 2000
Climate data: January 2001
Climate data: February 2001
Climate data: March 2001
Climate data: April 2001
Climate data: May 2001
Climate data: June 2001
Climate data: July 2001
Climate data: August 2001
Climate data: September 2001
Climate data: October 2001
Climate data: November 2001
Climate data: December 2001
Climate data: January 2002
Climate data: February 2002
Climate data: March 2002
Climate data: April 2002
Climate 

In [47]:
all_years.to_csv('all_years.csv')